Attemp to validate the training test

In [ ]:
%cd /content
!pip install -q roboflow
!pip install -q unzip
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
!pip install -e .

/content
fatal: destination path 'ultralytics' already exists and is not an empty directory.
/content/ultralytics
Obtaining file:///content/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.2.98-0.editable-py3-none-any.whl size=23034 sha256=69471cb38dea7a342f9c33f7945430e22d0a92f814ce7330e97de86d79ea83be
  Stored in directory: /tmp/pip-ephem-wheel-cache-27bsgtts/wheels/9a/cd/d5/95912172899f8ec640166ff6eef49156b1b00d6b2ade4a3cb1
Successfully built ultralytics
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.2.98
    Uninstalling ultralytics-8.2.98:
      Successfully uninstalled ultralytics-8.2.98


In [ ]:
%cd /content
!curl -L "https://universe.roboflow.com/ds/hrXfZEfvFj?key=6jtSKxauS0" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   910  100   910    0     0   2910      0 --:--:-- --:--:-- --:--:--  2916
100 80.5M  100 80.5M    0     0  40.8M      0  0:00:01  0:00:01 --:--:-- 65.7M
Archive:  roboflow.zip
replace README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: README.dataset.txt      
  inflating: README.roboflow.txt     
  inflating: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/-2021-10-22-162147_jpg.rf.07d45fa672da68beffb5e2c7a66c8aef.jpg  
 extracting: test/images/-2021-10-22-162147_jpg.rf.8a7ab59bf9aad49bf4a8bbd40beffa19.jpg  
 extracting: test/images/073_jpg.rf.ae32298fa1deb911bc8164d4e95ef1a4.jpg  
 extracting: test/images/092_jpg.rf.eb1cfe186ac845412c05014b885370a7.jpg  
 extracting: test/images/12_jpg.rf.af9bb0ad81293202c9b9422a3043fdc8.jpg  
 extracting: test/ima

In [ ]:
%cd /content
!wget http://14.225.204.171:9998/training_3_70_epoch_20.pt

/content
--2024-09-20 10:03:10--  http://14.225.204.171:9998/training_3_70_epoch_20.pt
Connecting to 14.225.204.171:9998... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6262799 (6.0M) [application/octet-stream]
Saving to: ‘training_3_70_epoch_20.pt’

training_3_70_epoch 100%[===================>]   5.97M   115KB/s    in 51s     

2024-09-20 10:04:02 (121 KB/s) - ‘training_3_70_epoch_20.pt’ saved [6262799/6262799]



In [ ]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
from contextlib import redirect_stdout
import logging
from google.colab.patches import cv2_imshow

# Define paths
model_path = '/content/training_3_70_epoch_20.pt'  # Path to your trained model
input_folder = '/content/test/images'           # Path to the folder containing images
label_folder = '/content/test/labels'           # Path to the folder containing ground truth labels
output_folder = '/content/output'          # Path to save the output images
csv_output_path = '/content/result/1st_model_evaluation_results.csv'  # Path to save the evaluation results CSV
txt_output_path = '/content/result/1st_model_final_conclusion.txt'    # Path to save the final conclusion TXT

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the YOLOv8 model
model = YOLO(model_path)

# Initialize lists to store evaluation results
results_data = []
image_with_detections_count = 0  # Counter for images with detections

# Function to parse label file into a list of ground truth classes
def parse_label_file(label_path):
    classes = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = parts[0]
            # Map class IDs to class names if necessary
            class_name = {
                '0': 'ball',
                '1': 'goalkeeper',
                '2': 'football-players',
                '3': 'referee',
            }.get(class_id, class_id)  # Default to class_id if not found
            classes.append(class_name)
    return classes

# Suppress logging messages
logging.getLogger().setLevel(logging.ERROR)

# Suppress detailed output from YOLO model predictions
with open(os.devnull, 'w') as fnull:
    with redirect_stdout(fnull):  # Suppress unwanted output
        for image_name in os.listdir(input_folder):
            # Full path to the image
            image_path = os.path.join(input_folder, image_name)

            # Check if the file is an image
            if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                continue

            # Perform detection
            results = model.predict(source=image_path, verbose=False)

            # Extract detected class names
            detected_classes = [model.names[int(cls)] for cls in results[0].boxes.cls]

            # If detections exist, increment the counter
            if detected_classes:
                image_with_detections_count += 1

            # Corresponding label file
            label_file = image_name.replace('.jpg', '.txt')  # Adjust extension if necessary
            label_path = os.path.join(label_folder, label_file)

            # Parse ground truth classes from label file
            if os.path.exists(label_path):
                gt_classes = parse_label_file(label_path)
            else:
                gt_classes = []

            # Compare detections with ground truth classes
            tp = len(set(detected_classes) & set(gt_classes))  # True Positives: correct class detections
            fp = len(detected_classes) - tp  # False Positives: incorrect class detections
            fn = len(gt_classes) - tp  # False Negatives: missed ground truth classes

            # Save results
            results_data.append({
                'Image': image_name,
                'True Positives': tp,
                'False Positives': fp,
                'False Negatives': fn,
                'Detected Classes': detected_classes,
                'Ground Truth Classes': gt_classes
            })

            # Save the output image with detections
            detected_img = results[0].plot()
            output_path = os.path.join(output_folder, f"detected_{image_name}")
            cv2.imwrite(output_path, detected_img)
            # Uncomment the line below if you want to display the image in Colab
            cv2_imshow(detected_img)

# Convert results to a DataFrame and save to CSV
df = pd.DataFrame(results_data)
df['Precision'] = df['True Positives'] / (df['True Positives'] + df['False Positives'])
df['Recall'] = df['True Positives'] / (df['True Positives'] + df['False Negatives'])
df['F1 Score'] = 2 * (df['Precision'] * df['Recall']) / (df['Precision'] + df['Recall'])

# Save evaluation results to CSV
df.to_csv(csv_output_path, index=False)

# Calculate overall evaluation metrics
total_tp = df['True Positives'].sum()
total_fp = df['False Positives'].sum()
total_fn = df['False Negatives'].sum()

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

# Save final conclusion to TXT file
with open(txt_output_path, 'w') as f:
    f.write(f"Overall Evaluation Metrics:\n")
    f.write(f"Total True Positives: {total_tp}\n")
    f.write(f"Total False Positives: {total_fp}\n")
    f.write(f"Total False Negatives: {total_fn}\n")
    f.write(f"Overall Precision: {overall_precision:.2f}\n")
    f.write(f"Overall Recall: {overall_recall:.2f}\n")
    f.write(f"Overall F1 Score: {overall_f1:.2f}\n")

print("Evaluation completed and results saved to:", csv_output_path)
print("Final conclusion saved to:", txt_output_path)

# Print the number of images with detections
print(f"Total number of images with detections: {image_with_detections_count}")